### Load libraries
```{r}

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import ndcg_score, average_precision_score
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

### Load and prepare data

In [ ]:
# Load datasets
anime_df = pd.read_csv('data/anime.csv', delimiter=',')
rating_df = pd.read_csv('data/rating.csv', delimiter=',')

### Load and preprocess data

In [ ]:
# --- 1. Load and Preprocess Data ---
# Load datasets
anime_df = pd.read_csv('data/anime.csv', delimiter=',')
rating_df = pd.read_csv('data/rating.csv', delimiter=',')

# Clean rating data by removing -1 and averaging duplicate ratings
rating_clean = rating_df[rating_df['rating'] != -1]
rating_clean = rating_clean.groupby(['user_id', 'anime_id']).agg({'rating': 'mean'}).reset_index()

# Merge anime and rating data
merged_df = pd.merge(rating_clean, anime_df, on='anime_id', how='inner')

# Prepare the data for embedding: Combine name, genre, type, and episodes
# Handle potential NaN values by filling them with empty strings
anime_df['name'] = anime_df['name'].fillna('')
anime_df['genre'] = anime_df['genre'].fillna('')
anime_df['type'] = anime_df['type'].fillna('')
anime_df['episodes'] = anime_df['episodes'].fillna('')

# Remove commas from the 'genre' column
anime_df['genre'] = anime_df['genre'].str.replace(',', ' ')


# Create a combined text string for each anime
anime_combined_features = []
for index, row in anime_df.iterrows():
    combined_text = f"{row['name']} genre: {row['genre']} type: {row['type']} episodes: {row['episodes']}"
    anime_combined_features.append(combined_text)

anime_id_to_name = dict(zip(anime_df['anime_id'], anime_df['name'])) # Still useful for displaying recommendations
name_to_anime_id = dict(zip(anime_df['name'], anime_df['anime_id'])) # Still useful for reverse lookup

### Load pretrained model and generate embeddings

In [ ]:
# --- 2. Load Pretrained Model and Generate Embeddings ---
def load_pretrained_model(model_name: str):
    """Loads a SentenceTransformer model."""
    return SentenceTransformer(model_name)

print("Loading SentenceTransformer model...")
model = load_pretrained_model('all-MiniLM-L6-v2') # Smaller, faster model for demonstration

print("Generating anime embeddings (this might take a while for large datasets)...")
# Use the combined features for embedding generation
anime_embeddings = model.encode(anime_combined_features, show_progress_bar=True)
print(f"Generated {anime_embeddings.shape[0]} embeddings of dimension {anime_embeddings.shape[1]}")

# Create a mapping from anime_id to its embedding
anime_id_to_embedding = {
    anime_df.loc[i, 'anime_id']: anime_embeddings[i]
    for i in range(len(anime_df))
}

### Calculate Cosine Similarity

In [ ]:
# --- 3. Calculate Cosine Similarity ---
# For efficient calculation, we will calculate similarity between all anime embeddings
print("Calculating cosine similarity matrix...")
anime_similarity_matrix = cosine_similarity(anime_embeddings)
print("Cosine similarity matrix calculated.")

# Create a mapping from index in similarity matrix back to anime_id
index_to_anime_id = {i: anime_df.loc[i, 'anime_id'] for i in range(len(anime_df))}
anime_id_to_index = {anime_df.loc[i, 'anime_id']: i for i in range(len(anime_df))}

### Implement Nearest Neighbors

In [ ]:
# --- 4. Implement Nearest Neighbors ---
def get_nearest_neighbors(anime_id: int, similarity_matrix: np.ndarray, top_n: int = 10):
    """
    Finds the top_n most similar animes for a given anime_id.
    Excludes the anime itself.
    """
    if anime_id not in anime_id_to_index:
        return []

    anime_idx = anime_id_to_index[anime_id]
    similarities = similarity_matrix[anime_idx]

    # Get indices of top_n+1 most similar items (including itself)
    # Use argsort to get indices, then reverse to get descending order of similarity
    top_similar_indices = similarities.argsort()[::-1][1:top_n + 1] # [1:] to exclude itself

    # Map indices back to anime_ids and their similarity scores
    nearest_neighbors = []
    for idx in top_similar_indices:
        neighbor_anime_id = index_to_anime_id[idx]
        neighbor_similarity = similarities[idx]
        nearest_neighbors.append((neighbor_anime_id, neighbor_similarity))

    return nearest_neighbors

# Sample usage:
# print("\nNearest neighbors for Anime A (ID 1):")
# print(get_nearest_neighbors(1, anime_similarity_matrix))


### Generate recommendations

In [ ]:
# --- 5. Generate Recommendations ---
def recommend_for_user(user_id: int, rating_data: pd.DataFrame, similarity_matrix: np.ndarray, top_k: int = 10):
    """
    Generates recommendations for a user based on their watched anime and similar items.
    This is a basic user-based collaborative filtering approach using item-item similarity.
    """
    user_watched_anime = rating_data[rating_data['user_id'] == user_id]
    if user_watched_anime.empty:
        print(f"User {user_id} has no watched anime data.")
        return []

    recommended_anime_scores = {}
    already_watched_anime_ids = set(user_watched_anime['anime_id'].tolist())

    for _, row in user_watched_anime.iterrows():
        watched_anime_id = row['anime_id']
        watched_rating = row['rating']

        # Get nearest neighbors for the watched anime
        neighbors = get_nearest_neighbors(watched_anime_id, similarity_matrix, top_n=50) # Get more neighbors to choose from

        for neighbor_anime_id, similarity_score in neighbors:
            # Only recommend if not already watched by the user
            if neighbor_anime_id not in already_watched_anime_ids:
                # Simple aggregation: sum of (similarity * user_rating_for_watched_item)
                # You can use more sophisticated weighted averages here
                if neighbor_anime_id not in recommended_anime_scores:
                    recommended_anime_scores[neighbor_anime_id] = 0
                recommended_anime_scores[neighbor_anime_id] += similarity_score * watched_rating

    # Sort recommendations by score in descending order
    sorted_recommendations = sorted(recommended_anime_scores.items(), key=lambda item: item[1], reverse=True)

    # Get the top_k recommendations, retrieving names
    final_recommendations = []
    for anime_id, score in sorted_recommendations[:top_k]:
        anime_name = anime_id_to_name.get(anime_id, f"Unknown Anime (ID: {anime_id})")
        final_recommendations.append({'anime_id': anime_id, 'name': anime_name, 'score': score})

    return final_recommendations

# Example usage:
# user_id_to_test = 1
# recommendations = recommend_for_user(user_id_to_test, rating_clean, anime_similarity_matrix, top_k=10)
# print(f"\nRecommendations for User {user_id_to_test}:")
# for rec in recommendations:
#     print(f"  - {rec['name']} (ID: {rec['anime_id']}) Score: {rec['score']:.2f}")



### Evaluation (using MAP@10 and NDCG@10)

In [ ]:
# --- 6. Evaluation (MAP@10 and NDCG@10) ---

# To evaluate, we need to split data into train and test sets.
# We'll use the train set to generate recommendations and the test set as ground truth.
train_rating, test_rating = train_test_split(rating_clean, test_size=0.2, random_state=42)

# Ensure all anime_ids in test_rating are present in anime_id_to_index for embedding lookups
# Filter out test ratings where anime_id is not in our processed anime_df
test_rating = test_rating[test_rating['anime_id'].isin(anime_df['anime_id'])]

def calculate_map_at_k(recommended_items, ground_truth_items, k=10):
    """
    Calculates Mean Average Precision at K (MAP@K).
    recommended_items: List of recommended item IDs (ordered by relevance).
    ground_truth_items: Set of relevant item IDs.
    """
    if not ground_truth_items:
        return 0.0

    relevant_count = 0
    precision_sum = 0.0

    for i, item_id in enumerate(recommended_items[:k]):
        if item_id in ground_truth_items:
            relevant_count += 1
            precision_sum += relevant_count / (i + 1)
    
    return precision_sum / min(len(ground_truth_items), k) if relevant_count > 0 else 0.0


def calculate_ndcg_at_k(recommended_items, ground_truth_items, k=10):
    """
    Calculates Normalized Discounted Cumulative Gain at K (NDCG@K).
    recommended_items: List of recommended item IDs (ordered by relevance).
    ground_truth_items: Set of relevant item IDs.
    """
    if not ground_truth_items:
        return 0.0

    # Create a relevance score list for NDCG calculation
    # 1 if item is relevant, 0 otherwise
    relevance = [1 if item_id in ground_truth_items else 0 for item_id in recommended_items[:k]]

    # For NDCG, we need a list of scores for actual and ideal ordering.
    # The `ndcg_score` function from sklearn expects a 2D array for y_true (relevance)
    # and y_score (predicted scores/rankings).
    # Since our recommendations are already ranked, we can use a simple array.
    # The ideal_dcg assumes all ground_truth_items are ranked perfectly at the top.
    
    # y_true should represent the relevance of each item in the *recommended list*.
    # y_score can be a list of dummy scores, as long as it preserves the ranking.
    # We'll use the relevance list itself for y_true and a descending range for y_score
    # to indicate the ranking.
    
    # Ensure y_true and y_score have the same length (k)
    y_true = np.asarray([relevance])
    # A simple descending score for the recommended items
    y_score = np.asarray([np.arange(k, 0, -1)])
    
    # If the length of relevance is less than k, pad with zeros
    if len(relevance) < k:
        relevance.extend([0] * (k - len(relevance)))
    
    try:
        # Use relevance scores directly for y_true for `ndcg_score`
        # and a proxy for the predicted scores (e.g., inverse of rank)
        return ndcg_score([relevance], [list(range(k, 0, -1))])
    except ValueError as e:
        # This error can occur if all y_true values are zero (no relevant items)
        # or if the dimensions don't match.
        # If no relevant items are found in recommendations, NDCG is 0.
        print(f"NDCG calculation error: {e}. Returning 0.0.")
        return 0.0


print("\nStarting evaluation for MAP@10 and NDCG@10...")
user_ids_to_evaluate = test_rating['user_id'].unique()
map_scores = []
ndcg_scores = []

# Filter users who have ratings in the training set
users_with_train_data = train_rating['user_id'].unique()
user_ids_for_evaluation = [uid for uid in user_ids_to_evaluate if uid in users_with_train_data]

if not user_ids_for_evaluation:
    print("No users with sufficient data in both train and test sets for evaluation. Please check your data split.")
else:
    for user_id in tqdm(user_ids_for_evaluation, desc="Evaluating users"):
        # Get ground truth from the test set
        ground_truth_anime_ids = set(test_rating[
            (test_rating['user_id'] == user_id) & (test_rating['rating'] >= 7) # Consider ratings >= 7 as "relevant"
        ]['anime_id'].tolist())

        if not ground_truth_anime_ids:
            continue # Skip users with no relevant items in test set

        # Generate recommendations based on the training data
        # Ensure 'rating_clean' is passed correctly, or 'train_rating' for generating recommendations
        recommendations = recommend_for_user(user_id, train_rating, anime_similarity_matrix, top_k=10)
        recommended_anime_ids = [rec['anime_id'] for rec in recommendations]

        # Calculate MAP@10
        map_scores.append(calculate_map_at_k(recommended_anime_ids, ground_truth_anime_ids, k=10))

        # Calculate NDCG@10
        ndcg_scores.append(calculate_ndcg_at_k(recommended_anime_ids, ground_truth_anime_ids, k=10))

    avg_map_at_10 = np.mean(map_scores) if map_scores else 0
    avg_ndcg_at_10 = np.mean(ndcg_scores) if ndcg_scores else 0

    print(f"\n Evaluation Results")
    print(f"Average MAP@10: {avg_map_at_10:.4f}")
    print(f"Average NDCG@10: {avg_ndcg_at_10:.4f}")

Loading SentenceTransformer model...


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

Generating anime embeddings (this might take a while for large datasets)...


Batches:   0%|          | 0/385 [00:00<?, ?it/s]

Generated 12294 embeddings of dimension 384
Calculating cosine similarity matrix...
Cosine similarity matrix calculated.

Starting evaluation for MAP@10 and NDCG@10...


Evaluating users: 100%|██████████| 61658/61658 [45:10<00:00, 22.75it/s]  


--- Evaluation Results ---
Average MAP@10: 0.0301
Average NDCG@10: 0.1643
